<a href="https://colab.research.google.com/github/josepeon/python_dad_class/blob/main/dad_apis_webscraping_hw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Homework III: API's and Webscraping

In [2]:
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup

#### Question 1

Using the LastFM API, look up your five favorite artists (using [here](https://www.last.fm/api/show/artist.getTopAlbums)) top albums and create a DataFrame of your results formatted as shown below:

| Artist | Album | Album Art Link |
| ------ | ------ | ------------  |
| Riff Raff | Pink Panther | www.lastfm.com/panther |
| Slayer | South of Heaven | www.lastfm.com/slay |

In [8]:
api_key = '6cac624f7267d913379d6d61963dbbad'
base_url = 'https://ws.audioscrobbler.com/2.0/'
artists = ['Kanye West', 'The Beatles', 'Bad Bunny', 'Silvio Rodriguez', 'Frank Ocean']
top_albums = []
for artist in artists:
    payload = {
        'method': 'artist.gettopalbums',
        'artist': artist,
        'api_key': api_key,
        'format': 'json',
        'limit': 1
    }

    r = requests.get(base_url, params=payload)
    data = r.json()
df_albums = pd.DataFrame(top_albums)
df_albums

""


#### Question 2

Using the LastFM API, look up tracks similar (using [this](https://www.last.fm/api/show/track.getSimilar)) to Quicksand's song Fazer.  Assemble the results in a DataFrame with the format given below:

| Artist | Song | Playcount | Match % | Duration |
| ------ | ------- | ------ | ------ | ---------- |
| quicksand | can opener | 98218 | 1.0 | 219 |

#### Question 3

arXiv is an open source space for academic papers to be published.  They have a freely accessible API [here](https://info.arxiv.org/help/api/user-manual.html#arxiv-api-users-manual).  In order to parse the responses, you will need to use the BeautifulSoup library and turn the text of the response into a soup object that is then searched.

Your objective is to write a function that takes in a search term and returns a `DataFrame` with the article date, title, authors, summary, and article url as columns of the `DataFrame`.

#### Question 4

The world bank has a Python wrapper for its api called `wbgapi`.  Examine the documentation [here](https://pypi.org/project/wbgapi/) and chose an endpoint(s) to query.  Find at least two endpoints of interest and create visualizations of this data.  Write a sentence or two about what you've found.

#### Question 5


Find an api of interest to you -- here is a list of some but feel free to google around [api list](https://github.com/public-apis/public-apis).  Ask a specific question that you want to use the data from the api to answer, make an appropriate request of the endpoints and do your best to provide an answer to your question asked.

-------
For example, maybe I'm interested in finding out recent artists similar to Rod Stewart.  I could use the LastFM api for this.  Perhaps you're interested in a lyrical analysis of Drake vs. Kendrick Lamar -- and want to compare the lexical diversity of different tracks; you can use the genius api for this.  Maybe I want to build an app to show a random cat picture with a dad joke.  The cat api and jokes api might work here.

#### Question 6

Use the `praw` api [here](https://praw.readthedocs.io/en/stable/) to extract posts from two subreddits of interest to you.  Create a DataFrame based on the posts that contains columns that you feel are appropriate based on the structure of each post.  At a minimum you want a title of each post, the body, and the subreddit title:

| Title | Text | Subreddit |
| ------- | ------ | ------ |
| If we could shrink ourselves down, <br/>at a certain size we would stop being <br/>able to hear anything because the sound <br/>waves would be too big for our tiny ear holes | None | showerthoughts |
| How do I actually get good at Python? | I wouldn’t call myself <br/>a complete new beginner in programming,<br/> I get the concepts. <br/>I know the basics ... <br/>how did you go from building simple scripts to building <br/>complex and big projects? | learnpython |

#### Question 7

Head over to [books to scrape](https://books.toscrape.com/).  Use `BeautifulSoup` to extract all the book titles and prices and assemble these in a DataFrame with two columns appropriately titled.

#### Question 8

Head over to the [Countries of the World](https://www.scrapethissite.com/pages/simple/) site.  Use requests and BeautifulSoup to scrape all countries (250) data and assemble as a DataFrame with columns:

| Country | Capital | Population | Area (km$^2$) | People/km$^2$ |
| -----  | -------- | ---------  | ------------  | ------------  |
| Andorra | Andorra la Vella | 84000 | 468.0 | 179.49 |